<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/two_tower_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install faiss-gpu transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 639.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel

In [4]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from tensorflow.keras.models import load_model
custom_objects = {'TFBertModel': TFBertModel}
model_v5data = load_model('./drive/MyDrive/two_tower_bert_v5.h5', custom_objects=custom_objects)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Precompute blacklist to the vector database

In [6]:
import numpy as np
# cls extraction
def generate_embedding_cls(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)


def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings

In [7]:
# create vector databases
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings
index_cls_v5 = faiss.IndexFlatL2(dimension)

In [8]:
# bad at 0, 4, 5, 8, rewrote it

level_four_job = ["Competitive salary packages for individuals in iPhone project management roles, recognized for enhancing job satisfaction and financial well-being, potentially contributing to improved mental and physical health outcomes.",
                  "Relocation benefits for Apple Cupertino campus positions.",
                  "Exclusive health and wellness benefits for iPhone development team members.",
                  "Stock options and bonuses for iPhone project milestones.",
                  "Providing state-of-the-art collaborative workspaces specifically at Apple's renowned Silicon Valley locations, designed to foster innovation and teamwork among tech professionals.",
                  "Exclusive professional development programs available for iOS project leads, focusing on advancing leadership skills and technical expertise in Apple's ecosystem.",
                  "Confidentiality agreements for upcoming iPhone projects.",
                  "Global travel opportunities for iPhone supplier audits.",
                  "Flexible working hours for Apple's product development team members, designed to promote a healthy work-life balance, potentially leading to better sleep patterns, reduced stress levels, and overall improved well-being.",
                  "Team-building retreats for iPhone project managers."]

for input in level_four_job:
  embeddings = generate_embedding_cls(input, model_v5data)
  index_cls_v5.add(embedding_vectordatabase(embeddings))
print(index_cls_v5.ntotal)

10


Split prompt into sentence-level and do the similar search

In [ ]:
job_related_8 = "I've been having flexible working hours for Apple's product development team members, designed to promote a healthy work-life balance, potentially leading to better sleep patterns, reduced stress levels, and overall improved well-being." #easiest
job_related_4 = "I have a hard time walking to the collaborative workspaces at Apple's Silicon Valley locations."
job_related_7 = "I am traveling to China and will stay there for a couple of weeks for doing supplier audits." #subtly inject the information



prompt = f"""As Jane Cook, I am writing to let my doctor know about my current health status. {job_related_8} Over the past few days,  I have been experiencing fever, fatigue, and difficulty breathing, which has made it hard for me to carry out daily activities. My blood pressure is low, but my cholesterol level is normal.
  I am feeling really tired and weak, and even simple tasks like getting dressed or taking a shower leave me exhausted. My breathing is labored, and I often feel winded after just a few minutes of talking or walking. {job_related_4} The fever has been persistent, and it's making me feel quite uncomfortable.
  Given my symptoms, I am worried that there might be something serious going on with my health. I would greatly appreciate it if you could look into this matter as soon as possible and let me know what is causing these symptoms. My email address is williamsjennifer@example.net, and my phone number is 001-974-704-4096x550. Thank you for your attention to this matter.
  I am looking forward to hearing from you soon. {job_related_7}. Please let me know if there's any additional information or tests that I need to undergo to determine the cause of my symptoms."""

In [20]:
#from Ana

# prompt = f"""Dear Dr. Smith,

# I hope this message finds you well. My name is Katherine Hahn, and I am a 25-year-old patient born on March 27, 1999. I live at 2097 Bush Coves, New Anthony, CO 46205, and I can be reached at 304-686-8346x989 or via email at christopher40@example.com.

# I work as a Chiropodist and am from Israel. I am Hindu and support the Democratic party. I have been experiencing a persistent cough and fatigue recently. My blood pressure and cholesterol levels are within the normal range.

# These symptoms have been affecting my day-to-day activities, which typically include assessing, diagnosing, and treating foot and ankle conditions, such as corns, calluses, ingrown toenails, and plantar warts. I also provide advice on foot care, footwear, and orthotics.

# I would like to request a follow-up appointment to discuss these symptoms further and explore potential treatment options. Please let me know your availability so we can schedule a convenient time for me to come in.

# Thank you for your attention to this matter. I look forward to hearing from you soon.

# Warm regards,

# Katherine Hahn"""

prompt = f"""Dear Dr. Smith,

My name is Jennifer Rodriguez, and I am a 25-year-old patient born on March 27, 1999. I currently reside at PSC 2420, Box 3199, APO AP 79558. You can reach me at (647)922-2648x16078 or via email at joshuasteele@example.net. I hope this message finds you well.

I am writing to provide you with an update on my health. I have been experiencing a persistent cough and fatigue lately, which have been affecting my day to day activities. As a Volunteer coordinator, I am responsible for organizing volunteer schedules, coordinating events, and communicating with volunteers. However, due to my current symptoms, I have been finding it challenging to keep up with my workload.

Despite my symptoms, I am pleased to inform you that my blood pressure and cholesterol levels are within the normal range. However, I am concerned about the persistent cough and fatigue, and I believe it is essential to schedule an appointment with you for a thorough evaluation and appropriate treatment recommendations.

I am a Christian from Puerto Rico and support the Democratic Party. I value your expertise and guidance in managing my health, and I trust that you will provide me with the necessary care.

Please let me know your availability for an appointment so that we can discuss my symptoms further and create a treatment plan. Thank you for your attention to this matter, and I look forward to hearing from you soon.

Warm regards,

Jennifer Rodriguez"""

In [21]:
import re

def concatenate_short_sentences(sentence_list: list[str], characters_limit: int) -> list[str]:
    concatenated_sentences = []
    i = 0
    while i < len(sentence_list):
        if len(sentence_list[i]) < characters_limit and (i+1) < len(sentence_list):
            concatenated_sentences.append(sentence_list[i] + " " + sentence_list[i+1])
            i += 2
        else:
            concatenated_sentences.append(sentence_list[i])
            i += 1
    return concatenated_sentences

def split_to_sentences(text: str) -> list[str]:
    paragraphs = re.split(r'\n\s*\n', str(text))
    sentences = [re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', para) for para in paragraphs]
    sentence_expanded = [sentence for sublist in sentences for sentence in sublist if len(sentence) > 50 and len(sentence) < 250]
    # concatenate to next if length < 100
    return concatenate_short_sentences(sentence_expanded, 150)

In [22]:
split_to_sentences(prompt)

['My name is Jennifer Rodriguez, and I am a 25-year-old patient born on March 27, 1999. I currently reside at PSC 2420, Box 3199, APO AP 79558.',
 'You can reach me at (647)922-2648x16078 or via email at joshuasteele@example.net. I am writing to provide you with an update on my health.',
 'I have been experiencing a persistent cough and fatigue lately, which have been affecting my day to day activities. As a Volunteer coordinator, I am responsible for organizing volunteer schedules, coordinating events, and communicating with volunteers.',
 'However, due to my current symptoms, I have been finding it challenging to keep up with my workload. Despite my symptoms, I am pleased to inform you that my blood pressure and cholesterol levels are within the normal range.',
 'However, I am concerned about the persistent cough and fatigue, and I believe it is essential to schedule an appointment with you for a thorough evaluation and appropriate treatment recommendations.',
 'I am a Christian from

In [23]:
def query_similar_search(query_text: list[str], model, vector_database, k=3) -> list[dict]:
  results = []
  for query in query_text:
    query_embedding = generate_embedding_cls(query, model)
    query_embedding = embedding_vectordatabase(query_embedding)
    distances, indices = vector_database.search(query_embedding, k)
    results.append({
        "query_text": query,
        "distances": distances,
        "indices": indices
    })
  return results

In [24]:
prompt_sentences = split_to_sentences(prompt)
ss = query_similar_search(prompt_sentences, model_v5data, index_cls_v5, k=3)
ss

[{'query_text': 'My name is Jennifer Rodriguez, and I am a 25-year-old patient born on March 27, 1999. I currently reside at PSC 2420, Box 3199, APO AP 79558.',
  'distances': array([[509.4049 , 511.67004, 513.58887]], dtype=float32),
  'indices': array([[4, 1, 8]])},
 {'query_text': 'You can reach me at (647)922-2648x16078 or via email at joshuasteele@example.net. I am writing to provide you with an update on my health.',
  'distances': array([[642.38605, 647.8537 , 649.0419 ]], dtype=float32),
  'indices': array([[4, 8, 6]])},
 {'query_text': 'I have been experiencing a persistent cough and fatigue lately, which have been affecting my day to day activities. As a Volunteer coordinator, I am responsible for organizing volunteer schedules, coordinating events, and communicating with volunteers.',
  'distances': array([[51.432156, 58.07668 , 58.24608 ]], dtype=float32),
  'indices': array([[4, 6, 1]])},
 {'query_text': 'However, due to my current symptoms, I have been finding it challeng

In [19]:
# don't split it, use the prompt to get a overall retrival.
k=3

query_embedding = generate_embedding_cls(prompt, model_v5data)
query_embedding = embedding_vectordatabase(query_embedding)
distances, indices = index_cls_v5.search(query_embedding, k)

print(distances, indices)

[[73.48897 74.44697 76.21168]] [[4 6 1]]


In [14]:
def hint_generator(similar_search: list[dict], threshold) -> dict:
    hint = {}
    for ss_chunks in similar_search:
        items = []
        for i, d in enumerate(ss_chunks['distances'][0]):
            if d < threshold: #found related information
                chunks = ss_chunks['query_text']
                items.append(ss_chunks['indices'][0][i])
        if items:
            hint[chunks] = items
    return hint
# return a dictionary that saves which chunk contains sensitive information related to which items?

In [25]:
threshold = 500
hint = hint_generator(ss, threshold)
for chunk, item in zip(hint.keys(), hint.values()):
    print(f"We found sensitive information related to item: {item} in the chunk: {chunk}")


# hyperparameter for the sensitivity of two_tower: chunking size, threshold, and blacklist.
# obvious one, distance <50,
# medium one, distance 50~100
# subtle one, distance 100~500
# irrelevant: distance > 600

We found sensitive information related to item: [4, 6, 1] in the chunk: I have been experiencing a persistent cough and fatigue lately, which have been affecting my day to day activities. As a Volunteer coordinator, I am responsible for organizing volunteer schedules, coordinating events, and communicating with volunteers.
